In [3]:
import numpy

In [5]:
import importlib.util
import os
import time

def get_last_access_time(package_name):
    '''
    Function that returns the spec for the package, walks the directory, 
    and then uses the maximum st_atime out of all files in that package.
    Note that,it still depends on the OS updating st_atime. It is possible that
    this timestamp does not accurately reflect “usage” of the package.
    '''
    spec = importlib.util.find_spec(package_name)
    if not spec or not spec.origin:
        return f"Could not find an importable spec for package '{package_name}'."
    
    # Convert the module’s main file to a directory path if needed
    package_path = os.path.dirname(spec.origin)
    if not os.path.exists(package_path):
        return "Package path not found."
    
    # Optionally, walk the entire directory to find the maximum last access time:
    last_atime = 0
    for root, _, files in os.walk(package_path):
        for file_ in files:
            filepath = os.path.join(root, file_)
            stats = os.stat(filepath)
            last_atime = max(last_atime, stats.st_atime)
    
    return time.ctime(last_atime) if last_atime else "No file stats found."

print("Numpy was last accessed on:", get_last_access_time("pandas"))


Numpy was last accessed on: Wed Jan  1 15:00:05 2025


In [10]:
import pkg_resources
import os
import time
import pandas as pd

def get_all_packages_last_access_time():
    """
    Returns a dictionary of {package_name: last_access_time_string},
    where 'last_access_time_string' can also be "Package folder not found."
    or an error message.
    """
    package_access_times = {}
    
    ## Iterate over all installed distributions in the environment
    for dist in pkg_resources.working_set:
        package_name = dist.project_name  # e.g., 'numpy'
        
        try:
            ## Attempt to find the on-disk directory of the package
            package_path = os.path.join(dist.location, 
                                        package_name.replace('-', '_')  ## Many packages use underscores in the folder
                                        )
            
            if os.path.exists(package_path):
                stats = os.stat(package_path)
                ## Convert st_atime to a readable string
                last_access = time.ctime(stats.st_atime)
            else:
                last_access = "Package folder not found."
            
            package_access_times[package_name] = last_access
        
        except Exception as e:
            package_access_times[package_name] = f"Error: {e}"
    
    return package_access_times

In [11]:
results = get_all_packages_last_access_time()

In [13]:
results = pd.DataFrame(list(results.items()), columns=["Package", "Last_Access_Time"])

In [15]:
results.head()

,Package,Last_Access_Time
0,GitPython,Package folder not found.
1,MarkupSafe,Wed Jan 1 15:00:05 2025
2,altair,Wed Jan 1 14:21:33 2025
3,annotated-types,Wed Jan 1 14:21:40 2025
4,anyio,Wed Jan 1 14:59:54 2025


In [ ]:
results.to_csv("packages_last_access.csv", index=False)